<a href="https://colab.research.google.com/github/madhukarwakhare/Hadoop-Colab/blob/main/Hadoop_Map_Reduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.1/hadoop-3.3.1.tar.gz

In [ ]:
!tar -xzvf hadoop-3.3.1.tar.gz

In [3]:
#copy  hadoop file to user/local
!cp -r hadoop-3.3.1/ /usr/local/

In [4]:
#To find the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"

In [ ]:
#Running Hadoop
!/usr/local/hadoop-3.3.1/bin/hadoop

In [ ]:
# Download 20newsgroups dataset available at http://qwone.com/~jason/20Newsgroups.

In [ ]:
!wget http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz

!tar -xzvf 20news-18828.tar.gz

# Hadoop Streaming

In [ ]:
!find / -name 'hadoop-streaming*.jar'

In [9]:
!chmod u+rwx /content/mapper.py
!chmod u+rwx /content/reducer.py

chmod: cannot access '/content/mapper.py': No such file or directory
chmod: cannot access '/content/reducer.py': No such file or directory


In [ ]:
!/usr/local/hadoop-3.3.1/bin/hadoop jar /usr/local/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar -input /content/20news-18828/alt.atheism/49960 -output /content/output -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

In [ ]:

!ls /content/output

part-00000  _SUCCESS


In [ ]:
!cat /content/output/part-00000

In [ ]:
!cat mapper.py

# -*- coding: utf-8 -*-
"""mapper.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1yCwGyMXJT2qt3_58aLOOiJXO0GIaPcJd
"""



import sys
import io
import re
import nltk
nltk.download('stopwords',quiet=True)
from nltk.corpus import stopwords
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

stop_words = set(stopwords.words('english'))
input_stream = io.TextIOWrapper(sys.stdin.buffer, encoding='latin1')
for line in input_stream:
  line = line.strip()
  line = re.sub(r'[^\w\s]', '',line)
  line = line.lower()
  for x in line:
    if x in punctuations:
      line=line.replace(x, " ") 

  words=line.split()
  for word in words: 
    if word not in stop_words:
      print('%s\t%s' % (word, 1))

In [ ]:
!cat reducer.py

# -*- coding: utf-8 -*-
"""reducer.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1YzJ-vUsO5VYCyMrfPMow3s2IdxXkyQ0i
"""

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    line=line.lower()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)
    try:
      count = int(count)
    except ValueError:
      #count was not a number, so silently
      #ignore/discard this line
      continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print ('%s\t%s' % (current_word, current_count))
        current_count 